In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Embedding, Activation, Flatten
from keras.optimizers import Adam, SGD
import numpy as np
import re
import preprocessing as preproc
import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import helpers
import models
from tqdm import tqdm
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
start_time = time.time()

In [3]:
train_data, y, test_data = helpers.get_processed_data(full_dataset=False)

In [4]:
np.unique(y)

array([0., 1.])

In [5]:
vectorizer = TfidfVectorizer(stop_words=None, ngram_range=(1,2), sublinear_tf=True, max_features=1000)
X = vectorizer.fit_transform(train_data).toarray()

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [16]:
optimizer = Adam(lr=0.03, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.001, amsgrad=False)

In [17]:
# optimizer = SGD(lr=0.04, momentum=0.08, decay=0.001, nesterov=True)

In [18]:
X.shape

(200000, 1000)

In [19]:
def create_model():
    model = Sequential()
    model.add(Dense(units=64, input_dim=X.shape[1], activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
    return model
model = create_model()

In [20]:
result = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32, verbose=2)

Train on 170000 samples, validate on 30000 samples
Epoch 1/20
 - 10s - loss: 0.4535 - acc: 0.7767 - val_loss: 0.4357 - val_acc: 0.7894
Epoch 2/20
 - 10s - loss: 0.4202 - acc: 0.7994 - val_loss: 0.4300 - val_acc: 0.7909
Epoch 3/20
 - 10s - loss: 0.4069 - acc: 0.8079 - val_loss: 0.4319 - val_acc: 0.7925
Epoch 4/20
 - 10s - loss: 0.3977 - acc: 0.8138 - val_loss: 0.4330 - val_acc: 0.7926
Epoch 5/20
 - 10s - loss: 0.3907 - acc: 0.8177 - val_loss: 0.4337 - val_acc: 0.7926
Epoch 6/20
 - 10s - loss: 0.3861 - acc: 0.8201 - val_loss: 0.4344 - val_acc: 0.7927
Epoch 7/20
 - 10s - loss: 0.3821 - acc: 0.8229 - val_loss: 0.4351 - val_acc: 0.7927
Epoch 8/20
 - 9s - loss: 0.3783 - acc: 0.8249 - val_loss: 0.4367 - val_acc: 0.7928
Epoch 9/20
 - 9s - loss: 0.3756 - acc: 0.8264 - val_loss: 0.4385 - val_acc: 0.7918
Epoch 10/20
 - 10s - loss: 0.3736 - acc: 0.8283 - val_loss: 0.4395 - val_acc: 0.7916
Epoch 11/20
 - 10s - loss: 0.3714 - acc: 0.8292 - val_loss: 0.4408 - val_acc: 0.7903
Epoch 12/20
 - 9s - loss:

In [14]:
# result = model.fit_generator(generator=helpers.batch_generator(X_train, y_train, 32),
#                    epochs=10, validation_data=(X_test,y_test),
#                     steps_per_epoch=int(X_train.shape[0]/32), verbose=1)

In [ ]:
np.mean(result.history['val_acc'])

In [ ]:
# a = batch_generator(X,y,32)

In [ ]:
# next(a)

400 features
Me 10000 data:
# 78.35 Adam:decay 0.002 lr 0.05 epochs 100
# 73 Adam:decay 0.02

Me 20000 data:
# 77.23 Adam:decay 0.002 lr 0.05 epochs 100

Me 200000 data:
# 78.22 Adam:decay 0.002 lr 0.05 epochs 100

Ola

 78.94% adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.001, amsgrad=False)
 
 79.40% adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.0001, amsgrad=False)
 
Features 1000

 81% adam = Adam(lr=0.03, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.0001, amsgrad=False)
  
 83 % adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.0001, amsgrad=False)

In [ ]:
lines1 = lines[:10000] + lines[100000:110000]
y1 = np.concatenate((y[:10000], y[100000:110000]), axis=0)
y1 = y1.tolist()
print(len(lines1))
print(len(y1))

In [ ]:
np.unique(y)

84.5% adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.001, amsgrad=False)

82.5 adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.005, amsgrad=False)

83.49 adam = Adam(lr=0.08, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.005, amsgrad=False)

# best model so far
adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.001, amsgrad=False)
model = Sequential()
model.add(Dense(units=64, input_dim=X.shape[1], activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.summary()